In [269]:
# Get libraries
import pandas as pd
import arcpy
import arcgis
import os
from arcgis.features import GeoAccessor, GeoSeriesAccessor

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import get_sun, AltAz, EarthLocation

In [270]:
theday = '2019-10-18'
outputdaysyntax = '10_18_2019'
#theday = arcpy.GetParameterAsText(0)
#if arcpy.Exists(theday):
#    print("Using day input")
#else: 
#    theday = '2019-05-29'
#    print("Using test day input")

originalroad = r"C:\Stories\glareanalysis\glare1\SampleRoads.gdb\Street_Centerlines"
    
    
#originalroad = arcpy.GetParameterAsText(1)
#if arcpy.Exists(originalroad):
#    print("Using road input")
#else: 
#    originalroad = r"C:\Stories\glareanalysis\glare1\SampleRoads.gdb\Street_Centerlines_small"
#    print("Using test road input")   
 
terrain = r"C:\Stories\glareanalysis\glare1\Terrain.tif"

#terrain = arcpy.GetParameterAsText(2)    
#if arcpy.Exists(terrain):
#    print("Using terrain input")
#else: 
#    terrain = r"C:\Stories\glareanalysis\glare1\Terrain2m_fallschurch.tif"
#    print("Using test terrain input")  




#########

# PART I USER INPUTS - SUN Times to examine and general sun location coordinate for entire study area


# Sun time/location inputs use a table of data for the times of day of SUN TABLE INPUTS

# input the day of the year we are examining, and the general centerpoint of the location area
#theday = '2019-05-29'
coor_lat = 38.86755435
coor_long = -77.16801577
grid_id = '29'  # just an identifier for the name of a grid cell, this is not used in any calculations
                # think of this as a national grid id maybe    

sunhorizontalangle_filter1 = 18 # max angle of sun to consider for calculation (
                #this is befire we factor in the rd angle, think of this as a filter)

#look at these 5 min intervals in morning and afternoon hours
myhours = ['05','06','07','08','09','18','19','20','21','22']
myminutes = ['00','05','10','15','20','25','30','35','40','45','50','55']

#######

#PART 2 USER INPUTS - Road Featureclass and terrain inputs

# USER INPUTS - ALL PASTED TO TOP existing inputs
#originalroad = r"C:\Stories\glareanalysis\glare1\SampleRoads.gdb\Street_Centerlines"
#terrain = r"C:\Stories\glareanalysis\glare1\Terrain_fallschurch.tif"
arcpy.env.workspace = r"C:\Stories\glareanalysis\glare1\glare1.gdb"

######

# PART 4 USER INPUTS - FILTERED ANGLES to find the areas where the sun in your face when driving

SunDelta_HorizontalAngle_Max = 8
SunDelta_VerticalAngle_Max = 8

#name of final output filtered featureclass
output_roadsfc_calc = "glare_on_" + outputdaysyntax

# End User inputs

########


In [271]:
originalroad

'C:\\Stories\\glareanalysis\\glare1\\SampleRoads.gdb\\Street_Centerlines'

In [272]:
###########


# PART I - CREATE SUN DATAFRAME TABLE (non geospatial stuff)

# uses astropy to find sun angles at a given place and several times in the morning and afternoon

# ["sun_id", "FullDayTimeText", "SolarAzimuth", "SolarElevation", "X", "Y", "GridID" ]

print (r"Creating sun table ...")

Creating sun table ...


In [273]:
# construct/create an array of the datetimestrings to examine and add as a column
mydaytime = []

for aa in range(len(myhours)):
    for bb in range(len(myminutes)):
        date_time_str = theday + ' ' + myhours[aa] + ':' + myminutes[bb] 
        mydaytime.append(date_time_str)

mydaytime       

['2019-10-18 05:00',
 '2019-10-18 05:05',
 '2019-10-18 05:10',
 '2019-10-18 05:15',
 '2019-10-18 05:20',
 '2019-10-18 05:25',
 '2019-10-18 05:30',
 '2019-10-18 05:35',
 '2019-10-18 05:40',
 '2019-10-18 05:45',
 '2019-10-18 05:50',
 '2019-10-18 05:55',
 '2019-10-18 06:00',
 '2019-10-18 06:05',
 '2019-10-18 06:10',
 '2019-10-18 06:15',
 '2019-10-18 06:20',
 '2019-10-18 06:25',
 '2019-10-18 06:30',
 '2019-10-18 06:35',
 '2019-10-18 06:40',
 '2019-10-18 06:45',
 '2019-10-18 06:50',
 '2019-10-18 06:55',
 '2019-10-18 07:00',
 '2019-10-18 07:05',
 '2019-10-18 07:10',
 '2019-10-18 07:15',
 '2019-10-18 07:20',
 '2019-10-18 07:25',
 '2019-10-18 07:30',
 '2019-10-18 07:35',
 '2019-10-18 07:40',
 '2019-10-18 07:45',
 '2019-10-18 07:50',
 '2019-10-18 07:55',
 '2019-10-18 08:00',
 '2019-10-18 08:05',
 '2019-10-18 08:10',
 '2019-10-18 08:15',
 '2019-10-18 08:20',
 '2019-10-18 08:25',
 '2019-10-18 08:30',
 '2019-10-18 08:35',
 '2019-10-18 08:40',
 '2019-10-18 08:45',
 '2019-10-18 08:50',
 '2019-10-18 

In [274]:
#create the sun table dataframe
#from https://chrisalbon.com/python/data_wrangling/pandas_create_column_with_loop/

#suntablefields = ["sun_id","FullDayTimeText", "SolarAzimuth", "SolarElevation", "X", "Y", "GridID" ]

raw_data = {'FullDayTimeText': mydaytime }
sun_df = pd.DataFrame(raw_data, columns = ['FullDayTimeText'])

#add these hard coded fields to all rows so we persist this through ti the final FC 
# in case someone cares to use different sunposisitons in the same calcuation and wants to know
#what rpows used which sun position and or reference grid ID, such as the national 1 km grid
sun_df['X'] = coor_long
sun_df['Y'] = coor_lat
sun_df['GridID'] = grid_id

#re our date is a date tupe field and 
# http://strftime.org/
sun_df['FullDayTimeText'] = pd.to_datetime(sun_df['FullDayTimeText'], format="%Y-%m-%d %H:%M")
sun_df = sun_df.astype({'GridID': int})

sun_df

#sun_df.dtypes

,FullDayTimeText,X,Y,GridID
0,2019-10-18 05:00:00,-77.168016,38.867554,29
1,2019-10-18 05:05:00,-77.168016,38.867554,29
2,2019-10-18 05:10:00,-77.168016,38.867554,29
3,2019-10-18 05:15:00,-77.168016,38.867554,29
4,2019-10-18 05:20:00,-77.168016,38.867554,29
...,...,...,...,...
115,2019-10-18 22:35:00,-77.168016,38.867554,29
116,2019-10-18 22:40:00,-77.168016,38.867554,29
117,2019-10-18 22:45:00,-77.168016,38.867554,29
118,2019-10-18 22:50:00,-77.168016,38.867554,29


In [275]:
#Calcuate solar elevation field and add it

def CalcSolarElevation(x,y,tt):
    
  # if you get a value error
  #ValueError: interpolating from IERS_Auto using predictive values that are more
  #than 30.0 days old.

    #Normally you should not see this error because this class
    #automatically downloads the latest IERS-A table.  Perhaps you are
    #offline?  If you understand what you are doing then this error can be
    #suppressed by setting the auto_max_age configuration variable to
    #``None``:

  #from astropy.utils.iers import conf
  #conf.auto_max_age = None

    
      
  utcoffset = -4*u.hour  # Eastern Daylight Time  
  sun_time = Time(tt) - utcoffset  #UTC time
  loc = EarthLocation(lon = x * u.deg, lat = y * u.deg)
  altaz = AltAz(obstime=sun_time, location=loc)
  altangle = get_sun(sun_time).transform_to(altaz).alt 
  return altangle.degree

#this works because we loop and get the value out - it is slow though
for i in sun_df.index:
    sun_df.at[i, 'SolarElevation'] = CalcSolarElevation(sun_df.at[i,'X'], 
                                                        sun_df.at[i, 'Y'], 
                                                        sun_df.at[i, 'FullDayTimeText'])
sun_df

,FullDayTimeText,X,Y,GridID,SolarElevation
0,2019-10-18 05:00:00,-77.168016,38.867554,29,-28.052448
1,2019-10-18 05:05:00,-77.168016,38.867554,29,-27.095822
2,2019-10-18 05:10:00,-77.168016,38.867554,29,-26.136636
3,2019-10-18 05:15:00,-77.168016,38.867554,29,-25.175137
4,2019-10-18 05:20:00,-77.168016,38.867554,29,-24.211568
...,...,...,...,...,...
115,2019-10-18 22:35:00,-77.168016,38.867554,29,-47.557889
116,2019-10-18 22:40:00,-77.168016,38.867554,29,-48.360920
117,2019-10-18 22:45:00,-77.168016,38.867554,29,-49.150104
118,2019-10-18 22:50:00,-77.168016,38.867554,29,-49.924544


In [276]:
# calc solar azimuth

def CalcSolarAzimuth(x,y,tt):
    
  utcoffset = -4*u.hour  # Eastern Daylight Time  
  sun_time = Time(tt) - utcoffset  #UTC time
  loc = EarthLocation(lon = x * u.deg, lat = y * u.deg)
  altaz = AltAz(obstime=sun_time, location=loc)
  azangle = get_sun(sun_time).transform_to(altaz).az
  return azangle.degree

#df['SolarAzimuth'] = CalcSolarAzimuth(df['X'], df['Y'], df['FullDayTimeText'])
for i in sun_df.index:
    sun_df.at[i, 'SolarAzimuth'] = CalcSolarAzimuth(sun_df.at[i,'X'], 
                                                    sun_df.at[i, 'Y'], 
                                                    sun_df.at[i, 'FullDayTimeText'])

sun_df
#sun_df.dtypes

,FullDayTimeText,X,Y,GridID,SolarElevation,SolarAzimuth
0,2019-10-18 05:00:00,-77.168016,38.867554,29,-28.052448,79.180473
1,2019-10-18 05:05:00,-77.168016,38.867554,29,-27.095822,80.057695
2,2019-10-18 05:10:00,-77.168016,38.867554,29,-26.136636,80.923902
3,2019-10-18 05:15:00,-77.168016,38.867554,29,-25.175137,81.779821
4,2019-10-18 05:20:00,-77.168016,38.867554,29,-24.211568,82.626151
...,...,...,...,...,...,...
115,2019-10-18 22:35:00,-77.168016,38.867554,29,-47.557889,303.812314
116,2019-10-18 22:40:00,-77.168016,38.867554,29,-48.360920,305.207185
117,2019-10-18 22:45:00,-77.168016,38.867554,29,-49.150104,306.641808
118,2019-10-18 22:50:00,-77.168016,38.867554,29,-49.924544,308.117902


In [277]:
# test to show all rows
import IPython.display

IPython.display.HTML(sun_df.to_html())

,FullDayTimeText,X,Y,GridID,SolarElevation,SolarAzimuth
0,2019-10-18 05:00:00,-77.168016,38.867554,29,-28.052448,79.180473
1,2019-10-18 05:05:00,-77.168016,38.867554,29,-27.095822,80.057695
2,2019-10-18 05:10:00,-77.168016,38.867554,29,-26.136636,80.923902
3,2019-10-18 05:15:00,-77.168016,38.867554,29,-25.175137,81.779821
4,2019-10-18 05:20:00,-77.168016,38.867554,29,-24.211568,82.626151
5,2019-10-18 05:25:00,-77.168016,38.867554,29,-23.246159,83.463569
6,2019-10-18 05:30:00,-77.168016,38.867554,29,-22.279136,84.292725
7,2019-10-18 05:35:00,-77.168016,38.867554,29,-21.310716,85.114250
8,2019-10-18 05:40:00,-77.168016,38.867554,29,-20.341110,85.928751
9,2019-10-18 05:45:00,-77.168016,38.867554,29,-19.370523,86.736819


In [278]:
#only keep rows where the elevation angle is less than than the max angle
sun_df = sun_df[sun_df.SolarElevation <= sunhorizontalangle_filter1] #this is currenlty 18, set above, before the road filter

#only keep rows above zero
sun_df = sun_df[sun_df.SolarElevation > 0 ]

#sun_df

In [279]:
#create list of sun-ids to add as a column from 1 - length

sun_df['sun_id'] = range(1,1+len(sun_df))

sun_df

#print (r"... created sun table ok. ")

,FullDayTimeText,X,Y,GridID,SolarElevation,SolarAzimuth,sun_id
30,2019-10-18 07:30:00,-77.168016,38.867554,29,0.908704,103.121621,1
31,2019-10-18 07:35:00,-77.168016,38.867554,29,1.854306,103.912616,2
32,2019-10-18 07:40:00,-77.168016,38.867554,29,2.796665,104.707892,3
33,2019-10-18 07:45:00,-77.168016,38.867554,29,3.735581,105.507873,4
34,2019-10-18 07:50:00,-77.168016,38.867554,29,4.670847,106.312985,5
35,2019-10-18 07:55:00,-77.168016,38.867554,29,5.602254,107.123655,6
36,2019-10-18 08:00:00,-77.168016,38.867554,29,6.529587,107.940312,7
37,2019-10-18 08:05:00,-77.168016,38.867554,29,7.452625,108.763389,8
38,2019-10-18 08:10:00,-77.168016,38.867554,29,8.371142,109.593321,9
39,2019-10-18 08:15:00,-77.168016,38.867554,29,9.284905,110.430547,10


In [280]:
# END PART I - CREATE SUN DATAFRAME TABLE (non geospatial stuff)

##################################


# PART 2 - This takes the raw roads and then adds the vertices and then 
#         creates the 3d segments with bearing and length attributes


print (r"Creating road segment table ...")

Creating road segment table ...


In [281]:
# USER INPUTS - ALL PASTED TO TOP existing inputs
#originalroad = r"C:\Stories\glareanalysis\glare1\SampleRoads.gdb\Street_Centerlines_small"
#terrain = r"C:\Stories\glareanalysis\glare1\Terrain2m_fallschurch.tif"
#arcpy.env.workspace = r"C:\Stories\glareanalysis\glare1\glare1.gdb"

# Set more environment settings
arcpy.env.outputCoordinateSystem = arcpy.Describe(terrain).spatialReference

#scratch featureclass files that are created on the fly
inputroad = "inputroad"
vertices2d = "vertices2d"
vertices3d = "vertices3d"
simpleline3d = "simpleline3d"
segments3d = "segments3d"

In [282]:
# make a working copy from original
arcpy.management.CopyFeatures(originalroad, inputroad)


<Result 'C:\\Stories\\glareanalysis\\glare1\\glare1.gdb\\inputroad'>

In [283]:
#add a field to preserve the objectid as a field
arcpy.management.AddField(inputroad, "RoadID", "LONG")
arcpy.management.CalculateField(inputroad, "RoadID", "!OBJECTID!", "PYTHON3")


<Result 'C:\\Stories\\glareanalysis\\glare1\\glare1.gdb\\inputroad'>

In [284]:

# generalize to remove most vertices
arcpy.edit.Generalize(inputroad, "5 Meters")


<Result 'C:\\Stories\\glareanalysis\\glare1\\glare1.gdb\\inputroad'>

In [285]:


#insert vertices every 200m
#arcpy.edit.Densify(inputroad, "DISTANCE", "200 Meters", "0.33 Feet", 10)
arcpy.edit.Densify(inputroad, "DISTANCE", "200 Meters")

<Result 'C:\\Stories\\glareanalysis\\glare1\\glare1.gdb\\inputroad'>

In [286]:
#make vertices
arcpy.management.FeatureVerticesToPoints(inputroad, vertices2d, "ALL")

<Result 'C:\\Stories\\glareanalysis\\glare1\\glare1.gdb\\vertices2d'>

In [287]:
#get the elevation of every vertice
arcpy.ddd.InterpolateShape(terrain, vertices2d, vertices3d, None, 1, "BILINEAR", "DENSIFY", 0, "EXCLUDE")



<Result 'C:\\Stories\\glareanalysis\\glare1\\glare1.gdb\\vertices3d'>

In [288]:
#create 3d roads from 3d vertices
arcpy.management.PointsToLine(vertices3d, simpleline3d, "RoadID", None, "NO_CLOSE")



AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

<Result 'C:\\Stories\\glareanalysis\\glare1\\glare1.gdb\\simpleline3d'>

In [289]:
#create short 3d segments from 3d roads
arcpy.management.SplitLine(simpleline3d, segments3d)



<Result 'C:\\Stories\\glareanalysis\\glare1\\glare1.gdb\\segments3d'>

In [290]:

#add GIS attributes
arcpy.management.AddGeometryAttributes(segments3d, "LENGTH_GEODESIC;LINE_BEARING;LINE_START_MID_END", "METERS")



AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

<Result 'C:\\Stories\\glareanalysis\\glare1\\glare1.gdb\\segments3d'>

In [291]:
print (r"... creating road table ok.")


#  END PART II - CREATE RD SEGMENTS WITH ATTRIBUTES





#############

# PART 3 - CREATE MASTER LARGE PANDAS COMPARISION TABLE 
# -  NOW WORK IN DATAFRAMES WITH OUR RD FC AND COMBINE OUR SUNTABLE To OUR ROADS
# - (NO USER INPUTS)

print (r"Creating large Pandas comparison dataframe ... ")

... creating road table ok.
Creating large Pandas comparison dataframe ... 


In [292]:
#create sdf
roadfc_df = pd.DataFrame.spatial.from_featureclass(segments3d, key=None)

In [293]:


def findRoadAngle(length_hypotenuse, upper_pt, lower_pt):
    length_opposite = upper_pt - lower_pt
    angleSine = math.degrees(math.sin(length_opposite/length_hypotenuse))
    return angleSine

In [294]:
#Add the AngleA, AngleB
for i in roadfc_df.index:
    roadfc_df.at[i, 'AngleA'] = findRoadAngle(roadfc_df.at[i,'LENGTH_GEO'], 
                                              roadfc_df.at[i, 'END_Z'], 
                                              roadfc_df.at[i, 'START_Z'])

for i in roadfc_df.index:
    roadfc_df.at[i, 'AngleB'] = findRoadAngle(roadfc_df.at[i,'LENGTH_GEO'], 
                                              roadfc_df.at[i, 'START_Z'], 
                                              roadfc_df.at[i, 'END_Z'])



In [295]:

# change column name as it is already calculated for BearingA
roadfc_df.rename(columns = {'BEARING':'BearingA'}, inplace = True)

#find BearingB the opposite bearing on a circle from north
def findOppositeBearingeAngle(x):
    a = x + 180
    if a > 360:
        a = a -360
    return a

for i in roadfc_df.index:
    roadfc_df.at[i, 'BearingB'] = findOppositeBearingeAngle(roadfc_df.at[i,'BearingA'])



In [296]:
roadfc_df

,OBJECTID,RoadID,LENGTH_GEO,BearingA,START_X,START_Y,START_Z,MID_X,MID_Y,MID_Z,END_X,END_Y,END_Z,SHAPE,AngleA,AngleB,BearingB
0,1,1,38.976377,231.295931,-8.589423e+06,4.703659e+06,107.0190,-8.589442e+06,4.703643e+06,105.69265,-8.589462e+06,4.703627e+06,104.3663,"{""hasZ"": true, ""paths"": [[[-8589422.523830935,...",-3.896494,3.896494,51.295931
1,2,2,87.920572,117.918209,-8.589423e+06,4.703659e+06,107.0190,-8.589373e+06,4.703632e+06,109.19090,-8.589323e+06,4.703606e+06,111.3628,"{""hasZ"": true, ""paths"": [[[-8589422.523830935,...",2.829601,-2.829601,297.918209
2,3,2,87.921026,117.918209,-8.589323e+06,4.703606e+06,111.3628,-8.589273e+06,4.703579e+06,113.79940,-8.589223e+06,4.703553e+06,116.2360,"{""hasZ"": true, ""paths"": [[[-8589322.778290486,...",3.174108,-3.174108,297.918209
3,4,3,43.429778,227.530498,-8.589561e+06,4.703731e+06,101.9118,-8.589581e+06,4.703712e+06,100.99120,-8.589602e+06,4.703694e+06,100.0706,"{""hasZ"": true, ""paths"": [[[-8589560.68491804, ...",-2.428320,2.428320,47.530498
4,5,4,121.526952,117.689717,-8.589561e+06,4.703731e+06,101.9118,-8.589492e+06,4.703695e+06,104.46540,-8.589423e+06,4.703659e+06,107.0190,"{""hasZ"": true, ""paths"": [[[-8589560.68491804, ...",2.407161,-2.407161,297.689717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1486,1487,1026,1.935296,284.902888,-8.592859e+06,4.706258e+06,108.8284,-8.592860e+06,4.706258e+06,108.85075,-8.592861e+06,4.706258e+06,108.8731,"{""hasZ"": true, ""paths"": [[[-8592858.828094749,...",1.323257,-1.323257,104.902888
1487,1488,1027,17.544380,240.983115,-8.591027e+06,4.704673e+06,91.6028,-8.591036e+06,4.704668e+06,91.37120,-8.591046e+06,4.704662e+06,91.1396,"{""hasZ"": true, ""paths"": [[[-8591026.500829745,...",-1.512526,1.512526,60.983115
1488,1489,1028,17.543739,134.500570,-8.591027e+06,4.704673e+06,91.6028,-8.591018e+06,4.704665e+06,91.56015,-8.591010e+06,4.704658e+06,91.5175,"{""hasZ"": true, ""paths"": [[[-8591026.500829745,...",-0.278579,0.278579,314.500570
1489,1490,1028,35.510124,97.878298,-8.591010e+06,4.704658e+06,91.5175,-8.590988e+06,4.704654e+06,91.47390,-8.590965e+06,4.704651e+06,91.4303,"{""hasZ"": true, ""paths"": [[[-8591010.415726472,...",-0.140698,0.140698,277.878298


In [297]:
#create a copy of the table and this will be our target df
#set initial value for a value of  sun id 1
temp_df = roadfc_df.copy()
temp_df['sun_id'] = 1
final_df = temp_df

# now concanenate the same table over and over for sun id 2 to X
for i in range(2, len(sun_df)):
    temp_df = roadfc_df.copy()
    temp_df['sun_id'] = i
        
    final_df = pd.concat([final_df, temp_df])
    
final_df

,OBJECTID,RoadID,LENGTH_GEO,BearingA,START_X,START_Y,START_Z,MID_X,MID_Y,MID_Z,END_X,END_Y,END_Z,SHAPE,AngleA,AngleB,BearingB,sun_id
0,1,1,38.976377,231.295931,-8.589423e+06,4.703659e+06,107.0190,-8.589442e+06,4.703643e+06,105.69265,-8.589462e+06,4.703627e+06,104.3663,"{""hasZ"": true, ""paths"": [[[-8589422.523830935,...",-3.896494,3.896494,51.295931,1
1,2,2,87.920572,117.918209,-8.589423e+06,4.703659e+06,107.0190,-8.589373e+06,4.703632e+06,109.19090,-8.589323e+06,4.703606e+06,111.3628,"{""hasZ"": true, ""paths"": [[[-8589422.523830935,...",2.829601,-2.829601,297.918209,1
2,3,2,87.921026,117.918209,-8.589323e+06,4.703606e+06,111.3628,-8.589273e+06,4.703579e+06,113.79940,-8.589223e+06,4.703553e+06,116.2360,"{""hasZ"": true, ""paths"": [[[-8589322.778290486,...",3.174108,-3.174108,297.918209,1
3,4,3,43.429778,227.530498,-8.589561e+06,4.703731e+06,101.9118,-8.589581e+06,4.703712e+06,100.99120,-8.589602e+06,4.703694e+06,100.0706,"{""hasZ"": true, ""paths"": [[[-8589560.68491804, ...",-2.428320,2.428320,47.530498,1
4,5,4,121.526952,117.689717,-8.589561e+06,4.703731e+06,101.9118,-8.589492e+06,4.703695e+06,104.46540,-8.589423e+06,4.703659e+06,107.0190,"{""hasZ"": true, ""paths"": [[[-8589560.68491804, ...",2.407161,-2.407161,297.689717,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1486,1487,1026,1.935296,284.902888,-8.592859e+06,4.706258e+06,108.8284,-8.592860e+06,4.706258e+06,108.85075,-8.592861e+06,4.706258e+06,108.8731,"{""hasZ"": true, ""paths"": [[[-8592858.828094749,...",1.323257,-1.323257,104.902888,23
1487,1488,1027,17.544380,240.983115,-8.591027e+06,4.704673e+06,91.6028,-8.591036e+06,4.704668e+06,91.37120,-8.591046e+06,4.704662e+06,91.1396,"{""hasZ"": true, ""paths"": [[[-8591026.500829745,...",-1.512526,1.512526,60.983115,23
1488,1489,1028,17.543739,134.500570,-8.591027e+06,4.704673e+06,91.6028,-8.591018e+06,4.704665e+06,91.56015,-8.591010e+06,4.704658e+06,91.5175,"{""hasZ"": true, ""paths"": [[[-8591026.500829745,...",-0.278579,0.278579,314.500570,23
1489,1490,1028,35.510124,97.878298,-8.591010e+06,4.704658e+06,91.5175,-8.590988e+06,4.704654e+06,91.47390,-8.590965e+06,4.704651e+06,91.4303,"{""hasZ"": true, ""paths"": [[[-8591010.415726472,...",-0.140698,0.140698,277.878298,23


In [298]:
# merge sun table to final_df
#from https://chrisalbon.com/python/data_wrangling/pandas_join_merge_dataframe/

finalwithsun_df = pd.merge(final_df, sun_df, on='sun_id')

finalwithsun_df

,OBJECTID,RoadID,LENGTH_GEO,BearingA,START_X,START_Y,START_Z,MID_X,MID_Y,MID_Z,...,AngleA,AngleB,BearingB,sun_id,FullDayTimeText,X,Y,GridID,SolarElevation,SolarAzimuth
0,1,1,38.976377,231.295931,-8.589423e+06,4.703659e+06,107.0190,-8.589442e+06,4.703643e+06,105.69265,...,-3.896494,3.896494,51.295931,1,2019-10-18 07:30:00,-77.168016,38.867554,29,0.908704,103.121621
1,2,2,87.920572,117.918209,-8.589423e+06,4.703659e+06,107.0190,-8.589373e+06,4.703632e+06,109.19090,...,2.829601,-2.829601,297.918209,1,2019-10-18 07:30:00,-77.168016,38.867554,29,0.908704,103.121621
2,3,2,87.921026,117.918209,-8.589323e+06,4.703606e+06,111.3628,-8.589273e+06,4.703579e+06,113.79940,...,3.174108,-3.174108,297.918209,1,2019-10-18 07:30:00,-77.168016,38.867554,29,0.908704,103.121621
3,4,3,43.429778,227.530498,-8.589561e+06,4.703731e+06,101.9118,-8.589581e+06,4.703712e+06,100.99120,...,-2.428320,2.428320,47.530498,1,2019-10-18 07:30:00,-77.168016,38.867554,29,0.908704,103.121621
4,5,4,121.526952,117.689717,-8.589561e+06,4.703731e+06,101.9118,-8.589492e+06,4.703695e+06,104.46540,...,2.407161,-2.407161,297.689717,1,2019-10-18 07:30:00,-77.168016,38.867554,29,0.908704,103.121621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34288,1487,1026,1.935296,284.902888,-8.592859e+06,4.706258e+06,108.8284,-8.592860e+06,4.706258e+06,108.85075,...,1.323257,-1.323257,104.902888,23,2019-10-18 18:15:00,-77.168016,38.867554,29,1.304739,256.336900
34289,1488,1027,17.544380,240.983115,-8.591027e+06,4.704673e+06,91.6028,-8.591036e+06,4.704668e+06,91.37120,...,-1.512526,1.512526,60.983115,23,2019-10-18 18:15:00,-77.168016,38.867554,29,1.304739,256.336900
34290,1489,1028,17.543739,134.500570,-8.591027e+06,4.704673e+06,91.6028,-8.591018e+06,4.704665e+06,91.56015,...,-0.278579,0.278579,314.500570,23,2019-10-18 18:15:00,-77.168016,38.867554,29,1.304739,256.336900
34291,1490,1028,35.510124,97.878298,-8.591010e+06,4.704658e+06,91.5175,-8.590988e+06,4.704654e+06,91.47390,...,-0.140698,0.140698,277.878298,23,2019-10-18 18:15:00,-77.168016,38.867554,29,1.304739,256.336900


In [299]:
# use this function to find the small angle azimuth angle between the sun and road azimuths
# ie, small angle beweek 355 and 5 is 10
# ie, small angle between 175 and 185 is 10
def findsmallangle(x,y):
    a = x - y
    a = (a + 180) % 360 - 180
    return abs(a)

#this finds the sun and road vertical angle difference
#vert_angle = sunangle - roadangle
def findsignedvertanglediff(x,y):
    a = x - y
    return a

In [300]:

# Create a new column called df.SolarAzimuthDeltaA 
finalwithsun_df['SolarAzimuthDeltaA'] = findsmallangle(finalwithsun_df['BearingA'], finalwithsun_df['SolarAzimuth'])

# Create a new column called df.SolarAzimuthDeltaB
finalwithsun_df['SolarAzimuthDeltaB'] = findsmallangle(finalwithsun_df['BearingB'], finalwithsun_df['SolarAzimuth'])


In [301]:
# find vert angle difference A
finalwithsun_df['AngleADelta'] =  findsignedvertanglediff(finalwithsun_df['SolarElevation'], finalwithsun_df['AngleA'])


# find vert angle difference B
finalwithsun_df['AngleBDelta'] =  findsignedvertanglediff(finalwithsun_df['SolarElevation'], finalwithsun_df['AngleB'])


In [302]:
print (r"Number of rows in dataframe = " + str(len(finalwithsun_df)))

Number of rows in dataframe = 34293


In [303]:
# End PART 3 - CREATE MASTER LARGE PANDAS COMPARISION TABLE 

print (r"Created large pandas dataframe  ok.")

#######################

# PART 4  - FILTER OUT RESULTS

# FILTER OUT ONLY THE GOOOD ROAD SEGMENTS WHERE THE HORIZONTAL AND VERTIVAL ANGLES ARE CLOSE TO THE ALLOWED

Created large pandas dataframe  ok.


In [304]:
# USer allowed maxes (no negatives allowed for vertical of courser)
#pasted to top of script
#SunDelta_HorizontalAngle_Max = 8
#SunDelta_VerticalAngle_Max = 8

In [305]:
#HORIZONTAL FILTER

#from https://chrisalbon.com/python/data_wrangling/pandas_selecting_rows_on_conditions/

# Create 2 variables with TRUE for each horizontal angles of the 
# delta (difference between rd and sun) compared to the max allowed if angle is less than allowed offset
#original query in Pro
# SolarAzimuthDeltaA <= %HorizontalSunDelta% Or SolarAzimuthDeltaB <= %HorizontalSunDelta%

GoodSunHorizontalAngleA = finalwithsun_df['SolarAzimuthDeltaA'] <= SunDelta_HorizontalAngle_Max
GoodSunHorizontalAngleB = finalwithsun_df['SolarAzimuthDeltaB'] <= SunDelta_HorizontalAngle_Max

GoodSunHorizontalAngle = GoodSunHorizontalAngleA | GoodSunHorizontalAngleB
#GoodSunHorizontalAngle

In [306]:
#VERTICAL FILTER

# Create 2 variables with TRUE for each vertical angles of the 
# delta (difference between rd and sun) compared to the max allowed if angle is less than allowed offset

#original query in Pro
# #(AngleADelta <= %VerticalSunDelta% And AngleADelta >= 0) Or (AngleBDelta <= %VerticalSunDelta% And AngleBDelta >= 0)
GoodSunVerticalAngleA_1 = finalwithsun_df['AngleADelta'] <= SunDelta_VerticalAngle_Max
GoodSunVerticalAngleA_2 = finalwithsun_df['AngleADelta'] > 0
GoodSunVerticalAngleA = GoodSunVerticalAngleA_1 & GoodSunVerticalAngleA_2

#GoodSunVerticalAngleA

In [307]:
GoodSunVerticalAngleB_1 = finalwithsun_df['AngleBDelta'] <= SunDelta_VerticalAngle_Max
GoodSunVerticalAngleB_2 = finalwithsun_df['AngleBDelta'] > 0
GoodSunVerticalAngleB = GoodSunVerticalAngleB_1 & GoodSunVerticalAngleB_2

#GoodSunVerticalAngleB

GoodSunVerticalAngle = GoodSunVerticalAngleA | GoodSunVerticalAngleB
#GoodSunVerticalAngle

In [308]:
#Now filter the dataframe with the two queries
finalwithsun_filtered_df = finalwithsun_df[GoodSunHorizontalAngle & GoodSunVerticalAngle]

# went from 6000+ rows to
print (r"Filtered large pandas dataframe  ok.")
print (r"Number of glare road segments = " + str(len(finalwithsun_filtered_df)))
finalwithsun_filtered_df

Filtered large pandas dataframe  ok.
Number of glare road segments = 1121


,OBJECTID,RoadID,LENGTH_GEO,BearingA,START_X,START_Y,START_Z,MID_X,MID_Y,MID_Z,...,FullDayTimeText,X,Y,GridID,SolarElevation,SolarAzimuth,SolarAzimuthDeltaA,SolarAzimuthDeltaB,AngleADelta,AngleBDelta
22,23,15,85.329874,106.974700,-8.589216e+06,4.703943e+06,112.6745,-8.589163e+06,4.703927e+06,113.71860,...,2019-10-18 07:30:00,-77.168016,38.867554,29,0.908704,103.121621,3.853080,176.146920,-0.493304,2.310711
26,27,19,54.403812,287.405479,-8.588352e+06,4.703957e+06,105.6645,-8.588385e+06,4.703968e+06,104.74410,...,2019-10-18 07:30:00,-77.168016,38.867554,29,0.908704,103.121621,175.716142,4.283858,2.846986,-1.029579
36,37,27,18.681425,102.651624,-8.589943e+06,4.704069e+06,97.5075,-8.589931e+06,4.704067e+06,97.67480,...,2019-10-18 07:30:00,-77.168016,38.867554,29,0.908704,103.121621,0.469997,179.530003,-0.117457,1.934864
43,44,33,122.391056,95.861120,-8.589412e+06,4.704082e+06,108.8708,-8.589334e+06,4.704074e+06,109.04940,...,2019-10-18 07:30:00,-77.168016,38.867554,29,0.908704,103.121621,7.260501,172.739499,0.741485,1.075922
50,51,36,57.716646,96.618485,-8.589486e+06,4.704091e+06,113.4696,-8.589449e+06,4.704086e+06,111.17020,...,2019-10-18 07:30:00,-77.168016,38.867554,29,0.908704,103.121621,6.503136,173.496864,5.469140,-3.651733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34268,1467,1006,83.572823,262.383790,-8.588435e+06,4.703566e+06,114.4991,-8.588488e+06,4.703559e+06,113.39115,...,2019-10-18 18:15:00,-77.168016,38.867554,29,1.304739,256.336900,6.046889,173.953111,2.823736,-0.214258
34273,1472,1011,28.885047,258.110140,-8.588503e+06,4.703548e+06,113.0904,-8.588521e+06,4.703544e+06,112.73160,...,2019-10-18 18:15:00,-77.168016,38.867554,29,1.304739,256.336900,1.773239,178.226761,2.728009,-0.118531
34276,1475,1014,32.065038,249.399699,-8.592699e+06,4.704222e+06,117.5624,-8.592718e+06,4.704215e+06,117.57855,...,2019-10-18 18:15:00,-77.168016,38.867554,29,1.304739,256.336900,6.937201,173.062799,1.247023,1.362454
34277,1476,1015,16.509494,257.842845,-8.592724e+06,4.704229e+06,117.3866,-8.592734e+06,4.704227e+06,117.36630,...,2019-10-18 18:15:00,-77.168016,38.867554,29,1.304739,256.336900,1.505945,178.494055,1.445640,1.163838


In [309]:
# create the feature class output called output_roadsfc_calc
#finalwithsun_df.spatial.to_featureclass(input_fgdb,"roadsfc_calc",overwrite=True) 

finalwithsun_filtered_df.spatial.to_featureclass(output_roadsfc_calc) # output_roadsfc_calc

'C:\\Stories\\glareanalysis\\glare1\\glare1.gdb\\glare_on_10_18_2019'

In [310]:
# DONE
print (r"Outputed featureclass " + output_roadsfc_calc + " ok.")

Outputed featureclass glare_on_10_18_2019 ok.
